In [9]:
import glob
import torch
from torch.utils import data
from PIL import Image
import numpy as np
from torchvision import transforms
import matplotlib.pyplot as plt
#test

In [10]:
#通过创建data.Dataset子类Mydataset来创建输入
class Mydataset(data.Dataset):
# 类初始化
    def __init__(self, root):
        self.imgs_path = root
# 进行切片
    def __getitem__(self, index):
        img_path = self.imgs_path[index]
        return img_path
# 返回长度
    def __len__(self):
        return len(self.imgs_path)


In [13]:
#使用glob方法来获取数据图片的所有路径
all_imgs_path = glob.glob(r'./home/lyc/doc/AML_LAB/imu_data/*.csv')#数据文件夹路径，根据实际情况更改！
#循环遍历输出列表中的每个元素，显示出每个图片的路径
for var in all_imgs_path:
    print(var)

#利用自定义类Mydataset创建对象weather_dataset
signature_dataset = Mydataset(all_imgs_path)
print(len(signature_dataset)) #返回文件夹中图片总个数
print(signature_dataset[12:15])#切片，显示第12至第十五张图片的路径
sinature_datalodaer = torch.utils.data.DataLoader(signature_dataset, batch_size=5) #每次迭代时返回五个数据
print(next(iter(sinature_datalodaer)))

species = ['False','True']
species_to_id = dict((c, i) for i, c in enumerate(species))
print(species_to_id)
id_to_species = dict((v, k) for k, v in species_to_id.items())
print(id_to_species)
all_labels = []
#对所有图片路径进行迭代
for img in all_imgs_path:
    # 区分出每个img，应该属于什么类别
    for i, c in enumerate(species):
        if c in img:
            all_labels.append(i)
print(all_labels) #得到所有标签


0
[]


StopIteration: 

In [ ]:
# 对数据进行转换处理
transform = transforms.Compose([
                transforms.Resize((240,320)), #做的第一步转换
                transforms.ToTensor() #第二步转换，作用：第一转换成Tensor，第二将图片取值范围转换成0-1之间，第三会将channel置前
])

class Mydatasetpro(data.Dataset):
# 类初始化
    def __init__(self, img_paths, labels, transform):
        self.imgs = img_paths
        self.labels = labels
        self.transforms = transform
# 进行切片
    def __getitem__(self, index):                #根据给出的索引进行切片，并对其进行数据处理转换成Tensor，返回成Tensor
        img = self.imgs[index]
        label = self.labels[index]
        pil_img = Image.open(img)                 #pip install pillow
        data = self.transforms(pil_img)
        return data, label
# 返回长度
    def __len__(self):
        return len(self.imgs)

BATCH_SIZE = 10
signature_dataset = Mydatasetpro(all_imgs_path, all_labels, transform)
sinature_datalodaer = data.DataLoader(
                            signature_dataset,
                            batch_size=BATCH_SIZE,
                            shuffle=True
)

imgs_batch, labels_batch = next(iter(sinature_datalodaer))
print(imgs_batch.shape)

plt.figure(figsize=(12, 8))
for i, (img, label) in enumerate(zip(imgs_batch[:6], labels_batch[:6])):
    img = img.permute(1, 2, 0).numpy()
    plt.subplot(2, 3, i+1)
    plt.title(id_to_species.get(label.item()))
    plt.imshow(img)
plt.show()#展示图片


In [ ]:
import pandas as pd 

#划分测试集和训练集
index = np.random.permutation(len(all_imgs_path))

all_imgs_path = np.array(all_imgs_path)[index]
all_labels = np.array(all_labels)[index]

#80% as train
s = int(len(all_imgs_path)*0.7)
print(s)

train_imgs = all_imgs_path[:s]
train_labels = all_labels[:s]
test_imgs = all_imgs_path[s:]
test_labels = all_labels[s:]





print(test_labels)
train_ds = Mydatasetpro(train_imgs, train_labels, transform) #TrainSet TensorData
test_ds = Mydatasetpro(test_imgs, test_labels, transform) #TestSet TensorData

print(train_ds)
train_imgs = pd.get_dummies(train_imgs)
test_imgs = pd.get_dummies(test_imgs)
#print(train_ds)

train_dl = data.DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)#TrainSet Labels
test_dl = data.DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=True)#TestSet Labels






## 训练

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18
from torch.utils.data import DataLoader
from torchvision import models
from torchsummary import summary
# 假设您的数据集已经准备好，包括训练集和验证集
# 这里仅作示例，您需要根据实际情况修改数据加载部分



In [ ]:
for i,j in train_dl:
    print(i.shape)
    break

In [ ]:



# 初始化ResNet模型
model = resnet18(pretrained=True)
model.conv1 = nn.Conv2d(4, 64, kernel_size=7, stride=2, padding=3, bias=False)
num_classes = 2
model.fc = nn.Linear(model.fc.in_features, num_classes)

# 设置优化器和损失函数
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# 训练模型
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(30):  # 假设训练10个epoch
    model.train()
    running_loss = 0.0
    for images, labels in train_dl:
        images, labels = images.to(device), labels.to(device)  # 将图像和标签分别移动到设备上
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels.to(torch.int64))
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/30], Loss: {running_loss / len(train_dl)}")



summary(model, input_size=(4,256,256), batch_size=-1, device='cuda')
# 验证模型


model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_dl:
        images, labels = images.to(device), labels.to(device)  # 将图像和标签分别移动到设备上
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Validation Accuracy: {100 * correct / total:.2f}%")


In [2]:
True_list=[]
False_list=[]
for i in range(1,11):
    True_list.append('AML_LAB/imu_data/True_%dxizhi.csv'%i)
    False_list.append('AML_LAB/imu_data/False_%dxizhi.csv'%i)

In [ ]:
for ori,vary in zip(True_list[:6],False_list[:6]):
    df = pd.read_csv(ori)
    # 提取所需的数据列
    x = df['roll']
    y = df['pitch']
    z = df['yaw']

    # 创建3D图形
    
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(18, 6))
    # 绘制散点图
    axes[0]=fig.add_subplot(1,2,1,projection='3d')
    axes[0].scatter(x, y, z, c='r', marker='o')
    axes[0].set_title('True')
    # 设置坐标轴标签
    axes[0].set_xlabel('Roll')
    axes[0].set_ylabel('Pitch')
    axes[0].set_zlabel('Yaw')
    df = pd.read_csv(vary)
    # 提取所需的数据列
    x = df['roll']
    y = df['pitch']
    z = df['yaw']
    axes[0]=fig.add_subplot(1,2,1,projection='3d')
    axes[0].scatter(x, y, z, c='b', marker='o')
    axes[0].set_title('True')
    # 设置坐标轴标签
    axes[0].set_xlabel('Roll')
    axes[0].set_ylabel('Pitch')
    axes[0].set_zlabel('Yaw')
    # axes[1]=fig.add_subplot(1,2,2,projection='3d')
    # axes[1].scatter(x, y, z, c='b', marker='o')
    # axes[1].set_title('False')
    # # 设置坐标轴标签
    # axes[1].set_xlabel('Roll')
    # axes[1].set_ylabel('Pitch')
    # axes[1].set_zlabel('Yaw')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


# 假设True_list和False_list包含文件路径列表

# 仅选择前6个文件进行示例
for ori, vary in zip(True_list[:], False_list[:]):
    # 创建一个新的图形
    fig = plt.figure(figsize=(18, 6))
    ax = fig.add_subplot(111, projection='3d')

    # 读取True数据并绘制散点图（红色）
    df_true = pd.read_csv(ori)
    x_true = df_true['roll']
    y_true = df_true['pitch']
    z_true = df_true['yaw']
    ax.scatter(x_true, y_true, z_true, c='r', marker='o', label='True')

    # 读取False数据并绘制散点图（蓝色）
    df_false = pd.read_csv(vary)
    x_false = df_false['roll']
    y_false = df_false['pitch']
    z_false = df_false['yaw']
    ax.scatter(x_false, y_false, z_false, c='b', marker='o', label='False')

    # 设置坐标轴标签
    ax.set_xlabel('Roll')
    ax.set_ylabel('Pitch')
    ax.set_zlabel('Yaw')

    # 添加图例
    ax.legend()

    # 显示图形
    plt.show()


In [4]:
for i in range(1,11):
    True_list.append('AML_LAB/imu_data/True_%dche.csv'%i)
    False_list.append('AML_LAB/imu_data/False_%dche.csv'%i)

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# 假设True_list和False_list包含文件路径列表

# 创建一个空的DataFrame来存储所有数据
data = pd.DataFrame()

# 加载True数据并添加标签
for ori in True_list[:]:
    df = pd.read_csv(ori)
    df['label'] = 1  # True数据标签为1
    data = pd.concat([data, df], ignore_index=True)

# 加载False数据并添加标签
for vary in False_list[:]:
    df = pd.read_csv(vary)
    df['label'] = 0  # False数据标签为0
    data = pd.concat([data, df], ignore_index=True)

# 提取特征和标签
X = data[['roll', 'pitch', 'yaw']]
y = data['label']

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)#, random_state=40)

# 创建并训练随机森林分类器
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

# 预测测试集
y_pred = rf_classifier.predict(X_test)

# 计算准确率
accuracy = accuracy_score(y_test, y_pred)
print("accuracy: ", accuracy)


accuracy:  0.6447244094488189


In [1]:
!git clone https://github.com/Lycho556/AML_LAB.git

Cloning into 'AML_LAB'...
